In [1]:
import numpy as np
import pandas as pd

from sklearn.utils.validation import check_is_fitted
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold

Преобразование категориальных переменных средним значением целевой переменной (а также другой статистикой, основанной на таргете) - популярный метод работы с признаки, обладающий большим потенциалом, особенно полезный для древовидных моделей. Метод прост: посчитаем среднее значение целевой перменной для каждой категории (для задач регрессии) или вероятность того, что наблюдение принадлежит одному из классов (для задач классификации), и будем использовать ее в качестве значения при данной категории.

В этом ноутбуке я покажу, как различные вариации `target-encoding` с различными регуляризациями сравниваются с более классическими подходами, вроде `One-Hot-Encoding`. Я также дам свои объяснения, интуиции и рекомендации по использованию этих методов.

## 1. Категориальные признаки с большим количеством уникальных значений

Для признаков с большим количеством разных категорий требуются разные подходы к преобразованию, чем для признаков с низким количеством категорий. `One-Hot-Encoding` создаст огромное количество столбцов и затруднит процесс отбора признаков в древовидных методах: `One-Hot-Encoding` переполнит все другие признаки и сделает такие признаки слишком непропорционально важными для модели. `Label-Encoding` будет сложно обработать модели из-за того, что непонятно насколько преобразованные значения будут случайными и сколько потребуется разбиений для достижения хорошего качества.

### 1.1 Взаимодействие признаков обычно приводит к большому количеству уникальных значений

Во многих случаях, хорошо подобранные взаимодействия признаков могут оказаться весьма полезными. Они помогают модели извлекать информацию из данных с меньшим количеством разбиений и улучшать ее производительность. Комбинируя несколько категориальных переменных или непрерывных переменных, мы обычно создаем новую функцию высокой мощности. Этот аспект демонстрирует еще большую потребность в правильной схеме кодирования.

## 2. Обзор популярных методов обработки категорий для древовидных моделей

### 2.1. One-Hot-Encoding

`One-Hot-Encoding` отображает каждую категорию в вектор в $R^{(n−1)}$ или $R^{(n−k)}$, где $n > k > 1$, если мы хотим пропустить некоторые категории. Каждый вектор содержит одну $1$, а все остальные его значения равны $0$. Эта кодировка обычно используется в линейных моделях и не лучший выбор для моделей деревьев.

#### Преимущества:

* Каждая категория должным образом отделена от других. Мы не делаем никаких предположений относительно отношений между категориями с таким подходом к преобразованию признаков, поэтому модель в этом смысле менее предвзята.

* Очень просто реализовать.

#### Недостатки:

* Для функций с высокой размерностью, создается много столбцов. Это значительно замедляет обучение, и если модель случайным образом выбирает часть признаков для каждого дерева или разбиения, то шансы присутствия в выборке только `OHE`-признаков искусственно увеличиваются, а шансы другой переменные, которые должны учитываться в разбиении / дереве, сокращаются. Это заставляет модель рассматривать `OHE`-признаки как более полезные, что не обязательно так.


* На каждом разбиении деревья могут отделять только одну категорию от других. Деревья должны помещать каждую категорию в отдельную ячейку, нет другого способа разделить один столбец с `OHE`, кроме как между 0 и 1. Это приводит к большему количеству разбиений, необходимых для достижения такой же точности, как и другие, более компактные кодировки. Это снова замедляет обучение и не позволяет деревьям объединять похожие категории в один бин, что может снизить качество модели.


### 2.2. Label and frequency encoding

`Label-Encoding` - это отображение каждой категории на некоторое число в $R^1$. Числа (метки) обычно выбираются таким образом, что не имеют или почти не имеют значения с точки зрения отношений между категориями. Таким образом, категории, закодированные с помощью чисел, которые находятся близко друг к другу (обычно) связаны так же, как категории, которые находятся далеко друг от друга.

__Частотное кодирование__ - это способ использования частот категорий в качестве меток. Это может помочь, если частота коррелирует с целевой переменной, а также может помочь модели понять, что категории с маленьким количеством примеров менее надежны, чем более крупные, особенно когда частотное кодирование используется параллельно с другим типом кодирования.


#### Преимущества (по сравнению с `One-Hot-Encoding`):

* Более быстрое обучение, чем с `One-Hot-Encoding`. Цифры в $R^1$ являются более компактными представлениями, чем векторы в $R^{(n − 1)}$, что приводит к меньшему количеству признаков для деревьев, что приводит к более быстрому обучению.


* Требуется меньше разбиений, значит модель более стабильная и простая. В отличие от `One-Hot-Encoding`, деревья могут разбивать несколько категорий одновременно (с одной горячей всегда 1).


* Легко реализовать.

#### Недостатки:

* Предвзятость. `Label-Encoding` реализован таким образом, что предполагает определенную упорядоченную взаимосвязь между категориями. На самом деле, из-за случайности присвоения лейблов такой связи не существует.


* Нелинейность по отношению к целевой переменной и большое количество сплитов. При каждом бинарном разбиении древовидным моделям необходимо найти значение переменной, которое как можно лучше отделяет таргет. Гораздо труднее это делать, когда признак и целевая переменная имеют почти нулевую линейную зависимость. Деревьям приходится делать много разбиений, чтобы разместить отдельные категории в отдельных листьях.


## 3. Mean-Target-Encoding

`Mean-Target-Encoding` можно рассматривать как способ кодирования, который используется для того, чтобы значения признака коррелировали с целевой переменной. Для каждой категории, мы вычисляем среднее значение целевой переменной в обучающей выборке.

$$label_c = p_c$$

$p_c$ - среднее значение целевой переменной для категории $c$. Мы __не используем__ тестовые данные для оценки целевой переменной по очевидной причине: мы должны относиться к тестовым данным так, как будто мы не знаем ответы для них.

#### Преимущества (по сравнению с `Label-Encoding`):

* Меньше разбиений, быстрее обучение. Деревья могут использовать линейные отношения между преобразованным значением категории и целевой переменной. Это особенно полезно при работе с категориальными функциями высокой мощности: модели сложно поместить каждую небольшую категорию в отдельный лист, но если множество небольших категорий можно поместить в один лист вместе на основе их среднего целевого значения, тогда деревья могут учиться намного быстрее.


* Меньше предвзятости, поскольку теперь преобразованный значений категорий имеют больше смысла: более близкие значения означают более близкие категории с точки зрения целевой переменной.

#### Недостатки:

* Сложнее построить и провести валидацию.


* Легко переоборудовать, если не используется регуляризация.


### 3.1 Переобучение при `Mean-Target-Encoding`

Когда мы имеем дело с признаками высокой мощности, многие категории содержится небольшое количество примеров. Многие из этих категорий будут выглядеть как отличные признаки для модели, хотя на самом деле это не так.

Рассмотрим задачу бинарной классификации, и предположим, что это распределение целевой переменной для какой-то категории совершенно случайное: </br>

$$ P(Y|category_1) = Bernoulli(0.5) $$ 
 
Это означает, что категория имеет 0 предсказательную силу (если рассматривать ее отдельно). Но, допустим, теперь у нас есть 5 примеров в обучающей выборке с этой категорией, какая значение вероятности будет выглядеть как хороший предиктор? Вероятность получить все пять единиц или все пять нулей составляет 0.0625. Итак, если у нас есть 100 категорий, подобных этой, с 5 примерами и 0 предсказательной силой, мы ожидаем, что по крайней мере 6 из них будут иметь цель из всех 0 или цель из всех единиц.

Затем, если мы добавим комбинации, в которых четыре значения целевой переменной совпадают, вероятность того, что это произойдет, составит 0,375! Более чем каждая третья категория с нулевой предсказательной силой при небольшом размере выборки будет выглядеть как достойный предиктор. Деревья поместят эти категории в отдельные листы и научатся предсказывать экстремальные значения для этих категорий. Затем, когда мы получим те же категории в тестовых данных, большинство из них не будет иметь одинакового целевого распределения, и прогнозы модели будут неверными.


### 3.2. Использование априорной вероятности для регуляризации

Самый простой метод регуляризации - сместить преобразованные значения редких категорий ближе к среднему значению целевой переменной по всему набору данных. Таким образом, мы надеемся, что модель будет с меньшей вероятностью изучать очень высокие или очень низкие вероятности для небольших категорий: их преобразованные значения теперь будут ближе к середине и будут своего рода смешением с категориями, которые имеют меньшее среднее целевое значение.

Теперь преобразование для каждой категории как в обучающих, так и в тестовых данных будет оцениваться следующим образом:

$$label_c = \frac{p_c * n_c + p_{global} * \alpha} {n_c + \alpha}$$

$p_c$ - среднее значение целевой переменной для данной категории;

$n_c$ - количество примеров обучающей выборки с данным значением категории;

$p_{global}$ - среднее значение целевой переменной по всему набору данных;

$\alpha$ - параметр регуляризации

Недостатки очевидны, кодирование по-прежнему является псевдопрерывным, каждая категория будет закодирована отдельным действительным числом, что по-прежнему позволяет модели помещать любую категорию в отдельный лист и устанавливать экстремальную вероятность для прогноза в этом листе. Модель все еще склонна к переобучению, просто теперь для переобучения ей нужно сделать больше разбиений в деревьях, поскольку преобразованные значения категорий ближе к середине, и не имеют экстремальных значений. Нам нужно, каким-то образом рандломизировать преобразований категорий, чтобы небольшие категории с эксприемальным значением целевой переменной было трудно поместить в отдельный лист, но, при этом, более крупные категории, которым мы доверяем, по-прежнему, будут иметь значения, близкие к среднего значению для них. 


#### 3.3. KFold регуляризация для `Mean-Target-Encoding`

Идея `KFold` регуляризации состоит в том, чтобы использовать только часть примеров для оценки среднего значения целевой переменной для каждой конкретной категории. Разделим данные на `k` фолдов и для каждого примера вычислим преобразованное значение, используя все фолды, кроме того, в котором находится данный пример. Мы можем использовать регуляризацию из пункта 3.2. `KFold` регуляризацией для обеспечения более надженых результатов.

Теперь преобразование категорий состоит из следующих шагов:

* Разделите обучающую выборку на k фолдов;


* Затем для каждого образца:

    * исключить фолд, в котором находится данный пример;

    * выполнить кодировку, используя уравнение выше

Обратите внимание, что `KFold` регуляризация выполняется только для обучающих данных, тестовые данные по-прежнему оцениваются на основе обучающих примеров, поскольку мы не должны знать ответы для тестовых данных и определенно не можем использовать их для оценки категорий. Я считаю, что наилучшее количество фолдов составляет от 3 до 6, в зависимости от того, какой степени рандомизации вы хотите добиться.


#### 3.4. Дополнительная регуляризация

Выполним перемешивание обучающей выборки. Будем двигаться сверху вниз, и для каждого примера выполним преобразования, используя среднее значение целевой переменной всех примеров, которые расположены до текущего. 

Таким образом, мы создаем еще большую рандомизацию преобразованных значений внутри каждой категории. Каждая пример в обучающей выбокре в каждой категории теперь оценивается на основе другой подвыборки. Меньшие категории получают большую рандомизацию, большие категории получают меньше рандомизации и большинство примеров закодированы значением, которое близко к среднему значению категории. Это очень полезное свойство, потому что переобучение происходит в основном из-за маленьких категорий.

In [8]:
class TargetEncoding(BaseEstimator, TransformerMixin):

    def __init__(self,
                 alpha: float = 0,
                 folds: int = 5,
                ):
        self.folds = folds
        self.alpha = alpha
        self.features = None
        self.cv = None

    def fit(self, X, y=None):
        self.features = {}
        self.cv = KFold(
            n_splits=self.folds, shuffle=True, random_state=27
        )
        global_mean = np.mean(y)

        for fold_number, (train_idx, valid_idx) in enumerate(self.cv.split(X, y), start=1):
            x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
            y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

            data = pd.DataFrame({"feature": x_train, "target": y_train})
            data = data.groupby(["feature"])["target"].agg([np.mean, np.size])
            data = data.reset_index()
            score = data["mean"] * data["size"] + global_mean * self.alpha
            score = score / (data["size"] + self.alpha)
            
            self.features[f"fold_{fold_number}"] = {
                key: value for key, value in zip(data["feature"], score)
            }

        return self

    def transform(self, X, y=None):
        check_is_fitted(self, "features")
        # TBD

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        x_transformed = X.copy(deep=True)

        for fold_number, (train_idx, valid_idx) in enumerate(self.cv.split(X, y), start=1):
            x_transformed.loc[valid_idx] = x_transformed.loc[valid_idx].map(
                self.features[f"fold_{fold_number}"]
            )
        return x_transformed

In [9]:
data = pd.read_csv(
    "../data/competition_data/train.csv"
)
features = pd.read_csv(
    "../data/competition_data/client_profile.csv"
)
data = data.merge(
    features, how="inner", on="APPLICATION_NUMBER"
)
data.head(n=2)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,0,Cash,M,1,157500.0,855000.0,25128.0,Secondary / secondary special,Married,...,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0
1,123526683,0,Cash,F,0,135000.0,1006920.0,42660.0,Higher education,Married,...,2.0,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0


In [11]:
encoder = TargetEncoding(alpha=10)
encoder.fit_transform(data["EDUCATION_LEVEL"], data["TARGET"])

0        0.0899775
1        0.0526385
2        0.0891614
3         0.088654
4        0.0897628
           ...    
89534    0.0897628
89535     0.088654
89536     0.088654
89537    0.0834552
89538    0.0904259
Name: EDUCATION_LEVEL, Length: 89539, dtype: object